In [70]:
import pandas as pd
import numpy as np
import pymongo

pd.set_option('display.max_columns', 500)

In [71]:
db_name = 'guide_data'
service_collection_name = 'service'
company_collection_name = 'company'
deals_collection_name = 'deal'

conn = pymongo.MongoClient("mongodb://{}:{}@{}:{}/{}?authSource=admin".format('mongoadmin', '12345', 'localhost', '27017', db_name))

In [72]:
service_collection = lambda c: c[db_name][service_collection_name]
company_collection = lambda c: c[db_name][company_collection_name]
deals_collection = lambda c: c[db_name][deals_collection_name]

In [73]:
companies = pd.DataFrame(company_collection(conn).find())
cat_cols = ['got_support_from', 'did_get_support', 'service', 'stage_of_development', 'msp_category', 'is_export', 'inno_cluster_member', 'skolcovo_member', 'is_inno_company', 'is_startup']
companies[cat_cols] = companies[cat_cols].astype('category')
companies.drop(columns=['_id'], inplace=True)
companies.replace(['', 'н.д.'], np.nan, inplace=True)

In [74]:
companies.shape

(7347, 23)

In [75]:
deals = pd.DataFrame(deals_collection(conn).find())
deals[['round', 'deal_type']] = deals[['round', 'deal_type']].astype('category')
deals.drop(columns=['_id'], inplace=True)
deals.replace(['', 'н.д.'], np.nan, inplace=True)
deals = deals.applymap(lambda x: np.nan if isinstance(x, list) and len(x) == 0 else x)

In [76]:
deals.shape

(4546, 25)

In [77]:
services = pd.DataFrame(service_collection(conn).find())
services.drop(columns=['_id'], inplace=True)
services.replace(['', 'н.д.'], np.nan, inplace=True)

In [78]:
services.shape

(498, 26)

In [79]:
deals.head(2)

,deal_date,startup_name,startup_inn,gov_fund,gov_fund_sum_dol,gov_fund_sum_rub,private_fund,private_fund_sum_dol,private_fund_sum_rub,corp_fund,corp_fund_sum_dol,corp_fund_sum_rub,corp_investor,corp_investor_sum_dol,corp_investor_sum_rub,business_angel,business_angel_sum_dol,business_angel_sum_rub,accelerator,accelerator_sum_dol,accelerator_sum_rub,final_price_dol,final_price_rub,round,deal_type
0,2017-01-01,leadza,7721379557,NaN,0.0,0.0,[фрии],251551.654455,15000000.0,NaN,0.000000,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,251551.654455,0.0,seed,buy
1,2017-01-01,трансинфотех,7714858580,NaN,0.0,0.0,NaN,0.000000,0.0,[коммит кэпитал],670804.411881,40000000.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,670804.411881,0.0,seed,buy


In [80]:
present_inns = set(deals['startup_inn']) & set(companies['inn'])
len(present_inns)

285

In [81]:
deals[deals['startup_inn'].apply(lambda x: x in present_inns)][['gov_fund', 'private_fund', 'corp_fund', 'corp_investor', 'business_angel', 'accelerator']].count()

gov_fund          181
private_fund      947
corp_fund         125
corp_investor     489
business_angel    472
accelerator       375
dtype: int64

In [82]:
# np.concatenate(deals.dropna(subset=['corp_investor'])['corp_investor'].values)

In [83]:
services.head(1)

,inn,name,type,startup_stage,market,services,technologies,type_of_ownership,investment_round,investition_from_dol,investition_to_dol,tech_focus,fund_total_rub,fund_total_dol,num_of_investments,num_of_exits,geography,study_format,num_of_people_in_company_from,num_of_people_in_company_to,num_of_participants,okved_main,okved_secondary,corp_stage,business_model,monetary_support
0,7703638456,da vinci capital,VentureFund,"[ранний рост, расширение]","[cybersecurity, transport & logistics, busines...",[инвестиции],"[ar/vr, big data, блокчейн, интернет вещей, ис...",частный,"[раунд а, раунд в, раунд с+]",1000000.0,50000000.0,[],25000.0,397.0,17.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
deals[deals['deal_type'] == 'buy'].head(1)

,deal_date,startup_name,startup_inn,gov_fund,gov_fund_sum_dol,gov_fund_sum_rub,private_fund,private_fund_sum_dol,private_fund_sum_rub,corp_fund,corp_fund_sum_dol,corp_fund_sum_rub,corp_investor,corp_investor_sum_dol,corp_investor_sum_rub,business_angel,business_angel_sum_dol,business_angel_sum_rub,accelerator,accelerator_sum_dol,accelerator_sum_rub,final_price_dol,final_price_rub,round,deal_type
0,2017-01-01,leadza,7721379557,NaN,0.0,0.0,[фрии],251551.654455,15000000.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,251551.654455,0.0,seed,buy


In [85]:
def group_deals_by_investor_type():
    res = (deals[deals['deal_type'] == 'buy'].melt(id_vars=['startup_inn', 'round'], 
               value_vars=['gov_fund', 'private_fund', 'corp_fund', 'corp_investor', 'business_angel', 'accelerator'])
        .dropna()
        .groupby(['startup_inn', 'round'])['variable'].apply(pd.value_counts)
        .unstack().reset_index()
        .where(lambda x: x['startup_inn'].apply(lambda i: i in present_inns))
        .dropna(subset=['startup_inn'])
        .fillna(0).set_index(['startup_inn', 'round'])
        .apply(lambda x: x / x.sum(), axis=1))

    return res

companies_and_share_of_deals_by_fund_type = group_deals_by_investor_type()
companies_and_share_of_deals_by_fund_type.head()

,,accelerator,business_angel,corp_fund,corp_investor,gov_fund,private_fund
startup_inn,round,,,,,,
1615013084,seed,1.0,0.0,0.0,0.0,0.0,0.0
1650350558,pre-seed,1.0,0.0,0.0,0.0,0.0,0.0
1650390180,pre-seed,1.0,0.0,0.0,0.0,0.0,0.0
1655449455,seed,0.0,0.0,0.0,0.0,0.0,1.0
1657238107,seed,0.0,1.0,0.0,0.0,0.0,0.0


In [86]:
companies_and_share_of_deals_by_fund_type.sum()

accelerator        61.166667
business_angel     72.266667
corp_fund          19.983333
corp_investor      72.933333
gov_fund           39.416667
private_fund      105.233333
dtype: float64

In [87]:
# companies_and_share_of_deals_by_fund_type.to_csv('type_classifier_target.csv')

In [65]:
(companies.where(lambda x: x['inn'].apply(lambda i: i in present_inns)).dropna(how='all')
).set_index('inn').to_csv('type_classifier_data.csv')

In [88]:
def try_concat(arr):
    try:
        return np.concatenate(arr)
    except Exception as e:
        return []

def match_company_and_investor():
    ff = (
        deals[deals['deal_type'] == 'buy']
        .groupby(['startup_inn', 'round'])[['gov_fund', 'private_fund', 'corp_fund', 'corp_investor', 'business_angel', 'accelerator']]
        .apply(lambda x: set(try_concat([i for i in x.values.reshape((-1, )) if isinstance(i, list)]))))

    investor_and_company = pd.DataFrame()

    for row in services.to_dict(orient='records'):
        inv_name = row['name']
        fit = ff.where(ff.apply(lambda x: inv_name in x)).dropna(how='all')
        if fit.empty is False:
    #         print("Match: ", fit.shape, row)
            investor_and_company = investor_and_company.append(pd.DataFrame({k: [v] * fit.shape[0] for k, v in row.items()}, index=fit.index))
    return investor_and_company.rename(columns=lambda x: '{}__investor'.format(x)).merge(
        companies.rename(columns=lambda x: '{}__company'.format(x)), left_on='startup_inn', right_on='inn__company')

In [90]:
company_and_its_investor = match_company_and_investor()

In [91]:
company_and_its_investor['type__investor'].value_counts()

VentureFund          376
ProgressInstitute     15
Accelerator           12
Name: type__investor, dtype: int64

In [92]:
company_and_its_investor.head(3)

,inn__investor,name__investor,type__investor,startup_stage__investor,market__investor,services__investor,technologies__investor,type_of_ownership__investor,investment_round__investor,investition_from_dol__investor,investition_to_dol__investor,tech_focus__investor,fund_total_rub__investor,fund_total_dol__investor,num_of_investments__investor,num_of_exits__investor,geography__investor,study_format__investor,num_of_people_in_company_from__investor,num_of_people_in_company_to__investor,num_of_participants__investor,okved_main__investor,okved_secondary__investor,corp_stage__investor,business_model__investor,monetary_support__investor,inn__company,name__company,got_support_from__company,did_get_support__company,service__company,foundation_date__company,tech_focus__company,stage_of_development__company,market__company,technology__company,business_model__company,main_okved__company,okved_secondary__company,msp_category__company,is_export__company,inno_cluster_member__company,skolcovo_member__company,is_inno_company__company,is_startup__company,current_profit__company,current_profit_tax__company,current_revenue__company,type__company
0,7703638456,da vinci capital,VentureFund,"[ранний рост, расширение]","[cybersecurity, transport & logistics, busines...",[инвестиции],"[ar/vr, big data, блокчейн, интернет вещей, ис...",частный,"[раунд а, раунд в, раунд с+]",1000000.0,50000000.0,[],25000.0,397.0,17.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7704794345,"ооо ""геттакси рус""",карта инновационных решений,да,добавлено - карта инновационных решений,2011-11-03,[транспорт и логистика],расширение,[transport & logistics],[],[b2c],62.09,"[82.99, 46.19, 63.11.1, 47.91.2, 69.20.2, 46.1...",NaN,нет,да,нет,да,нет,-23.0,-15.0,1507.0,Company
1,7728461088,new industry ventures,VentureFund,"[ранний рост, расширение, посевная]",[cleantech],[инвестиции],"[3d моделирование, big data, интернет вещей, н...",частный,"[seed, раунд а, раунд в]",500000.0,3000000.0,[],4000.0,62.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7714762021,"ооо ""скайер ит""",московский акселератор,нет,московский акселератор,2008-12-23,[недвижимость и строительство],ранний рост,[proptech],[беспилотники],[b2b],72.19,"[62.09, 85.42, 62.02, 63.11.1, 30.30.11, 63.11...",юл микро,нет,да,да,да,нет,-16450.0,0.0,19127.0,Company
2,7728461088,new industry ventures,VentureFund,"[ранний рост, расширение, посевная]",[cleantech],[инвестиции],"[3d моделирование, big data, интернет вещей, н...",частный,"[seed, раунд а, раунд в]",500000.0,3000000.0,[],4000.0,62.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7714762021,"ооо ""радуга""",карта инновационных решений,да,добавлено - карта инновационных решений,2008-12-23,[недвижимость и строительство],ранний рост,[proptech],[беспилотники],[b2b],72.19,"[62.09, 85.42, 62.02, 63.11.1, 30.30.11, 63.11...",юл микро,нет,да,да,да,нет,-16450.0,0.0,19127.0,Company


In [93]:
company_and_its_investor.to_csv('company_and_its_investor.csv')

In [117]:
services.where(services['name'].apply(lambda x: x in te)).dropna(how='all')

,inn,name,startup_stage,market,services,technologies,type_of_ownership,investment_round,investition_from_dol,investition_to_dol,tech_focus,fund_total_rub,fund_total_dol,num_of_investments,num_of_exits,geography,study_format,num_of_people_in_company_from,num_of_people_in_company_to,num_of_participants,okved_main,okved_secondary,corp_stage,business_model,monetary_support
0,7703638456,da vinci capital,"[ранний рост, расширение]","[cybersecurity, transport & logistics, busines...",[инвестиции],"[ar/vr, big data, блокчейн, интернет вещей, ис...",частный,"[раунд а, раунд в, раунд с+]",1000000.0,50000000.0,[],25000.0,397.0,17.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7728461088,new industry ventures,"[ранний рост, расширение, посевная]",[cleantech],[инвестиции],"[3d моделирование, big data, интернет вещей, н...",частный,"[seed, раунд а, раунд в]",500000.0,3000000.0,[],4000.0,62.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,addventure,"[ранний рост, расширение]","[hrtech, consumer goods & services, foodtech, ...",[инвестиции],"[3d моделирование, ar/vr, big data, аддитивные...",частный,"[раунд а, раунд в, раунд с+]",1000000.0,10000000.0,[],0.0,0.0,45.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,admitad invest,"[ранний рост, посевная]","[business software, advertising & marketing, e...","[инвестиции, консультационная поддержка по раз...","[ar/vr, искусственный интеллект и машинное обу...",корпоративный,"[seed, раунд а]",200000.0,2000000.0,[],0.0,0.0,14.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,altair capital,"[ранний рост, расширение, посевная]","[edtech, insurancetech, business software, e-c...","[инвестиции, консультационная поддержка по раз...","[блокчейн, искусственный интеллект и машинное ...",частный,"[seed, раунд а, раунд с+]",100000.0,10000000.0,[],35000.0,500.0,271.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,0,smarthub,"[идея, посевная]","[agrotech, cybersecurity, edtech, transport & ...","[инвестиции, консультационная поддержка по раз...","[3d моделирование, ar/vr, big data, аддитивные...",частный,"[pre-seed, seed]",0.0,0.0,[],0.0,0.0,46.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,9703026834,dv capital,"[ранний рост, идея, посевная]","[agrotech, cybersecurity, edtech, transport & ...","[инвестиции, консультационная поддержка по раз...","[3d моделирование, ar/vr, big data, аддитивные...",частный,"[pre-seed, seed, раунд а]",0.0,0.0,[],0.0,0.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,7703425673,российский фонд прямых инвестиций,[расширение],NaN,"[инвестиции, обучение (образовательные програм...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[инвестиции]
485,7736004350,фонд содействия инновациям,[любая],NaN,"[финансовая поддержка (субсидии, гранты и т.п.)]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[субсидии/гранты]
